In [1]:
#Selenium爬蟲(網頁開啟、截圖)
!pip install selenium
#Spotify API載入
!pip install spotipy

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm
import math
import time

# 設置 Spotify API 憑證
client_id = '89696e4a61eb4965b17a34c4bf147476'
client_secret = '999d7cac18b74ebabf73a997d67c950e'

In [25]:
# 認證
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# 讀取 CSV 檔案
songlist = pd.read_csv('./realpaperdata.csv',encoding='cp1252')

songlist

,song,artist,year released,bpm,nrgy,dnce,dB,live,val,dur,...,pop,neg,neu,pos,Likes,Views,Comments,rank,follow,artist pop
0,STARSTRUKK (feat. Katy Perry),3OH!3,2009,140.000,81.0,61.0,-6.000,23.00,23.0,203.000,...,70,0.081,0.723,0.197,864726.0,167313865.0,57062.0,NaN,981372.0,NaN
1,My First Kiss (feat. Ke$ha),3OH!3,2010,138.000,89.0,68.0,-4.000,36.00,83.0,192.000,...,68,0.084,0.741,0.175,418889.0,73222906.0,37327.0,NaN,981372.0,NaN
2,I Need A Dollar,Aloe Blacc,2010,95.000,48.0,84.0,-7.000,9.00,96.0,243.000,...,72,0.062,0.751,0.187,975824.0,109037087.0,24807.0,NaN,709908.0,NaN
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,2010,93.000,87.0,66.0,-4.000,4.00,38.0,180.000,...,80,0.000,1.000,0.000,3721021.0,693937290.0,159408.0,NaN,2045444.0,NaN
4,Nothin' on You (feat. Bruno Mars),B.o.B,2010,104.000,85.0,69.0,-6.000,9.00,74.0,268.000,...,79,0.076,0.760,0.164,2815481.0,451007446.0,76751.0,NaN,2045444.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,exes,Tate McRae,2023,136.965,56.9,83.8,-6.324,10.20,55.1,159.400,...,56,0.062,0.674,0.264,381224.0,31929611.0,10372.0,NaN,5472304.0,NaN
1722,QLONA,KAROL G,2023,169.925,75.6,84.2,-7.409,8.71,42.1,172.798,...,79,0.016,0.903,0.081,929658.0,192651825.0,22275.0,NaN,47501572.0,NaN
1723,LOVE AGAIN,The Kid LAROI,2023,107.001,39.8,66.2,-6.691,11.10,46.8,145.850,...,69,0.073,0.638,0.289,357630.0,21994310.0,14198.0,NaN,5364413.0,NaN
1724,Feather,Sabrina Carpenter,2023,123.510,68.6,78.7,-4.370,9.27,83.6,185.553,...,91,0.085,0.705,0.210,994652.0,57454041.0,23747.0,NaN,9501759.0,NaN


In [24]:
# 設置 Selenium Chrome driver
options=Options()#透過Options物件進行chrome的設定
options.chrome_executable_path = "./chrome.exe"

#建立driver物件實體，用程式操作瀏覽器運作
driver=webdriver.Chrome(options=options)

# 初始化計數器
noselect = []

def search_artist_url(artist_name):
    search_url = f"https://open.spotify.com/search/{artist_name}/artists"
    driver.get(search_url)
    time.sleep(3)  # 等待頁面加載
    try:
        artist_link = driver.find_element(By.XPATH, '//a[contains(@href, "/artist/")]')
        return artist_link.get_attribute('href')
    except Exception as e:
        print(f"Could not find artist URL for {artist_name} using Selenium: {e}")
        return None

def get_artist_followers(artist_id):
    try:
        artist_info = sp.artist(artist_id)
        return artist_info['followers']['total']
    except Exception as e:
        print(f"Error fetching followers for artist ID {artist_id} using Spotify API: {e}")
        return None

with tqdm(total=len(songlist)) as pbar:
    for i in range(len(songlist)):
        # 只處理特徵值為虛值的數據
        if math.isnan(songlist['follow'][i]):
            # 去除問號
            if '?' in songlist['song'][i]:
                songlist['song'][i] = songlist['song'][i].replace('?', '')
            if '?' in songlist['artist'][i]:
                songlist['artist'][i] = songlist['artist'][i].replace('?', '')

            artist_name = songlist['artist'][i]
            artist_url = search_artist_url(artist_name)
            
            if artist_url:
                artist_id = artist_url.split('/')[-1]
                followers = get_artist_followers(artist_id)
                
                if followers is not None:
                    songlist.at[i, 'follow'] = followers
                else:
                    noselect.append(i)
            else:
                noselect.append(i)

            pbar.update(1)
        else:
            pbar.update(1)

# 保存結果到新的 CSV 檔案
output_file_path = './realpaperdata.csv'
songlist.to_csv(output_file_path, index=False, encoding='cp1252')
print(f'Saved artist follower counts to {output_file_path}')

driver.quit()

100%|██████████| 1726/1726 [00:03<00:00, 468.32it/s]

Could not find artist URL for Axwell /\ Ingrosso using Selenium: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@href, "/artist/")]"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6FE1E1F52+60322]
	(No symbol) [0x00007FF6FE15CEC9]
	(No symbol) [0x00007FF6FE017EBA]
	(No symbol) [0x00007FF6FE067676]
	(No symbol) [0x00007FF6FE06773C]
	(No symbol) [0x00007FF6FE0AE967]
	(No symbol) [0x00007FF6FE08C25F]
	(No symbol) [0x00007FF6FE0ABC80]
	(No symbol) [0x00007FF6FE08BFC3]
	(No symbol) [0x00007FF6FE059617]
	(No symbol) [0x00007FF6FE05A211]
	GetHandleVerifier [0x00007FF6FE4F94AD+3301629]
	GetHandleVerifier [0x00007FF6FE5436D3+3605283]
	GetHandleVerifier [0x00007FF6FE539450+3563680]
	GetHandleVerifier [0x00007FF6FE294326+790390]
	(No symbol) [0x00007FF6FE16750F]
	(No symb

In [26]:
# 設置 Selenium Chrome driver
options=Options()#透過Options物件進行chrome的設定
options.chrome_executable_path = "./chrome.exe"

#建立driver物件實體，用程式操作瀏覽器運作
driver=webdriver.Chrome(options=options)

# 初始化計數器
noselect = []

def search_artist_url(artist_name):
    search_url = f"https://open.spotify.com/search/{artist_name}/artists"
    driver.get(search_url)
    time.sleep(3)  # 等待頁面加載
    try:
        artist_link = driver.find_element(By.XPATH, '//a[contains(@href, "/artist/")]')
        return artist_link.get_attribute('href')
    except Exception as e:
        print(f"Could not find artist URL for {artist_name} using Selenium: {e}")
        return None

def get_artist_popularity(artist_id):
    try:
        artist_info = sp.artist(artist_id)
        return artist_info['popularity']
    except Exception as e:
        print(f"Error fetching popularity for artist ID {artist_id} using Spotify API: {e}")
        return None

with tqdm(total=len(songlist)) as pbar:
    for i in range(len(songlist)):
        # 只處理特徵值為虛值的數據
        if math.isnan(songlist['artist pop'][i]):
            # 去除問號
            if '?' in songlist['song'][i]:
                songlist['song'][i] = songlist['song'][i].replace('?', '')
            if '?' in songlist['artist'][i]:
                songlist['artist'][i] = songlist['artist'][i].replace('?', '')

            artist_name = songlist['artist'][i]
            artist_url = search_artist_url(artist_name)
            
            if artist_url:
                artist_id = artist_url.split('/')[-1]
                popularity = get_artist_popularity(artist_id)
                
                if popularity is not None:
                    songlist.at[i, 'artist pop'] = popularity
                else:
                    noselect.append(i)
            else:
                noselect.append(i)

            pbar.update(1)
        else:
            pbar.update(1)

# 保存結果到新的 CSV 檔案
output_file_path = './realpaperdataspot.csv'
songlist.to_csv(output_file_path, index=False, encoding='cp1252')
print(f'Saved artist popularity to {output_file_path}')

driver.quit()

# 初始化計數器
noselect = []

def search_artist_url(artist_name):
    search_url = f"https://open.spotify.com/search/{artist_name}/artists"
    driver.get(search_url)
    time.sleep(3)  # 等待頁面加載
    try:
        artist_link = driver.find_element(By.XPATH, '//a[contains(@href, "/artist/")]')
        return artist_link.get_attribute('href')
    except Exception as e:
        print(f"Could not find artist URL for {artist_name} using Selenium: {e}")
        return None

def get_artist_popularity(artist_id):
    try:
        artist_info = sp.artist(artist_id)
        return artist_info['popularity']
    except Exception as e:
        print(f"Error fetching popularity for artist ID {artist_id} using Spotify API: {e}")
        return None

with tqdm(total=len(songlist)) as pbar:
    for i in range(len(songlist)):
        # 只處理特徵值為虛值的數據
        if math.isnan(songlist['artist pop'][i]):
            # 去除問號
            if '?' in songlist['song'][i]:
                songlist['song'][i] = songlist['song'][i].replace('?', '')
            if '?' in songlist['artist'][i]:
                songlist['artist'][i] = songlist['artist'][i].replace('?', '')

            artist_name = songlist['artist'][i]
            artist_url = search_artist_url(artist_name)
            
            if artist_url:
                artist_id = artist_url.split('/')[-1]
                popularity = get_artist_popularity(artist_id)
                
                if popularity is not None:
                    songlist.at[i, 'artist pop'] = popularity
                else:
                    noselect.append(i)
            else:
                noselect.append(i)

            pbar.update(1)
        else:
            pbar.update(1)

# 保存結果到新的 CSV 檔案
output_file_path = './realpaperdataspot.csv'
songlist.to_csv(output_file_path, index=False, encoding='cp1252')
print(f'Saved artist popularity to {output_file_path}')

driver.quit()

  0%|          | 0/1726 [00:00<?, ?it/s]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 1/1726 [00:06<3:03:05,  6.37s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 2/1726 [00:11<2:37:45,  5.49s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 3/1726 [00:15<2:22:51,  4.97s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 4/1726 [00:20<2:19:55,  4.88s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 5/1726 [00:25<2:23:53,  5.02s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 6/1726 [00:29<2:15:16,  4.72s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 7/1726 [00:34<2:17:04,  4.78s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  0%|          | 8/1726 [00:40<2:2

  4%|▍         | 67/1726 [07:26<2:40:58,  5.82s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 68/1726 [07:30<2:25:23,  5.26s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 69/1726 [07:35<2:19:31,  5.05s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 70/1726 [07:39<2:16:04,  4.93s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 71/1726 [07:43<2:07:32,  4.62s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 72/1726 [07:47<2:01:49,  4.42s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 73/1726 [07:53<2:14:52,  4.90s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         | 74/1726 [07:57<2:07:27,  4.63s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  4%|▍         |

  8%|▊         | 133/1726 [12:55<2:01:18,  4.57s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 134/1726 [13:00<2:01:05,  4.56s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 135/1726 [13:04<1:56:52,  4.41s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 136/1726 [13:08<1:53:16,  4.27s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 137/1726 [13:12<1:55:33,  4.36s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 138/1726 [13:16<1:52:12,  4.24s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 139/1726 [13:21<1:53:51,  4.30s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊         | 140/1726 [13:25<1:53:26,  4.29s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
  8%|▊  

 12%|█▏        | 199/1726 [18:04<2:02:40,  4.82s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 200/1726 [18:08<1:59:10,  4.69s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 201/1726 [18:13<1:59:36,  4.71s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 202/1726 [18:18<1:58:48,  4.68s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 203/1726 [18:22<1:58:37,  4.67s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 204/1726 [18:27<1:58:23,  4.67s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 205/1726 [18:32<1:58:01,  4.66s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏        | 206/1726 [18:36<1:57:48,  4.65s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 12%|█▏ 

 15%|█▌        | 265/1726 [23:27<1:55:24,  4.74s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 15%|█▌        | 266/1726 [23:32<1:55:23,  4.74s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 15%|█▌        | 267/1726 [23:36<1:54:15,  4.70s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 16%|█▌        | 268/1726 [23:41<1:54:17,  4.70s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 16%|█▌        | 269/1726 [23:46<1:53:39,  4.68s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 16%|█▌        | 270/1726 [23:50<1:49:08,  4.50s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 16%|█▌        | 271/1726 [23:54<1:49:57,  4.53s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 16%|█▌        | 272/1726 [24:00<2:00:50,  4.99s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 16%|█▌ 

...

 93%|█████████▎| 1599/1726 [2:04:25<09:48,  4.64s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1600/1726 [2:04:29<09:32,  4.54s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1601/1726 [2:04:33<09:24,  4.52s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1602/1726 [2:04:37<08:50,  4.28s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1603/1726 [2:04:41<08:27,  4.13s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1604/1726 [2:04:46<08:39,  4.26s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1605/1726 [2:04:50<08:46,  4.35s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 93%|█████████▎| 1606/1726 [2:04:55<08:49,  4.41s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


 96%|█████████▋| 1662/1726 [2:09:03<04:45,  4.46s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 96%|█████████▋| 1663/1726 [2:09:07<04:38,  4.42s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 96%|█████████▋| 1664/1726 [2:09:12<04:41,  4.54s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 96%|█████████▋| 1665/1726 [2:09:17<04:38,  4.56s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 97%|█████████▋| 1666/1726 [2:09:21<04:32,  4.54s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 97%|█████████▋| 1667/1726 [2:09:26<04:25,  4.50s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 97%|█████████▋| 1668/1726 [2:09:30<04:20,  4.49s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
 97%|█████████▋| 1669/1726 [2:09:34<04:13,  4.45s/it]Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


Saved artist popularity to ./realpaperdataspot.csv


 43%|████▎     | 746/1726 [00:16<00:21, 45.79it/s]


MaxRetryError: HTTPConnectionPool(host='localhost', port=54975): Max retries exceeded with url: /session/a8c6d5fde0f24f4635c3842281587660/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002123D80E8C0>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。'))

In [27]:
songlist

,song,artist,year released,bpm,nrgy,dnce,dB,live,val,dur,...,pop,neg,neu,pos,Likes,Views,Comments,rank,follow,artist pop
0,STARSTRUKK (feat. Katy Perry),3OH!3,2009,140.000,81.0,61.0,-6.000,23.00,23.0,203.000,...,70,0.081,0.723,0.197,864726.0,167313865.0,57062.0,NaN,981372.0,62.0
1,My First Kiss (feat. Ke$ha),3OH!3,2010,138.000,89.0,68.0,-4.000,36.00,83.0,192.000,...,68,0.084,0.741,0.175,418889.0,73222906.0,37327.0,NaN,981372.0,62.0
2,I Need A Dollar,Aloe Blacc,2010,95.000,48.0,84.0,-7.000,9.00,96.0,243.000,...,72,0.062,0.751,0.187,975824.0,109037087.0,24807.0,NaN,709908.0,62.0
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,2010,93.000,87.0,66.0,-4.000,4.00,38.0,180.000,...,80,0.000,1.000,0.000,3721021.0,693937290.0,159408.0,NaN,2045444.0,71.0
4,Nothin' on You (feat. Bruno Mars),B.o.B,2010,104.000,85.0,69.0,-6.000,9.00,74.0,268.000,...,79,0.076,0.760,0.164,2815481.0,451007446.0,76751.0,NaN,2045444.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,exes,Tate McRae,2023,136.965,56.9,83.8,-6.324,10.20,55.1,159.400,...,56,0.062,0.674,0.264,381224.0,31929611.0,10372.0,NaN,5472304.0,82.0
1722,QLONA,KAROL G,2023,169.925,75.6,84.2,-7.409,8.71,42.1,172.798,...,79,0.016,0.903,0.081,929658.0,192651825.0,22275.0,NaN,47501572.0,88.0
1723,LOVE AGAIN,The Kid LAROI,2023,107.001,39.8,66.2,-6.691,11.10,46.8,145.850,...,69,0.073,0.638,0.289,357630.0,21994310.0,14198.0,NaN,5364413.0,78.0
1724,Feather,Sabrina Carpenter,2023,123.510,68.6,78.7,-4.370,9.27,83.6,185.553,...,91,0.085,0.705,0.210,994652.0,57454041.0,23747.0,NaN,9501759.0,90.0


In [28]:
songlist.to_csv('realpaperdata.csv', index=False,encoding='cp1252')